<a href="https://colab.research.google.com/github/GontseGee/AI-mashup/blob/main/CV_JOB_MATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Import  dataset

In [3]:
df = pd.read_csv('UpdatedResumeDataSet.csv')

In [4]:
df.head()


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [5]:
df['Category'].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [6]:
df['Resume'][1]

'Education Details \r\nMay 2013 to May 2017 B.E   UIT-RGPV\r\nData Scientist \r\n\r\nData Scientist - Matelabs\r\nSkill Details \r\nPython- Exprience - Less than 1 year months\r\nStatsmodels- Exprience - 12 months\r\nAWS- Exprience - Less than 1 year months\r\nMachine learning- Exprience - Less than 1 year months\r\nSklearn- Exprience - Less than 1 year months\r\nScipy- Exprience - Less than 1 year months\r\nKeras- Exprience - Less than 1 year monthsCompany Details \r\ncompany - Matelabs\r\ndescription - ML Platform for business professionals, dummies and enthusiasts.\r\n60/A Koramangala 5th block,\r\nAchievements/Tasks behind sukh sagar, Bengaluru,\r\nIndia                               Developed and deployed auto preprocessing steps of machine learning mainly missing value\r\ntreatment, outlier detection, encoding, scaling, feature selection and dimensionality reduction.\r\nDeployed automated classification and regression model.\r\nlinkedin.com/in/aditya-rathore-\r\nb4600b146        

our data needs to be preprocessed to remove unwanted charecters and stop words that wont add value to the model.

#Data Pre-processing

In [7]:
import re
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Compare CV to Role match

removing unwanted charecters in our cvs


In [8]:
def preprocess_text(text_to_clean):
    # Remove URLs
    cleaned_text = re.sub('http\S+\s*', ' ', text_to_clean)
    # Remove RT and cc
    cleaned_text = re.sub('RT|cc', ' ', cleaned_text)
    # Remove hashtags
    cleaned_text = re.sub('#\S+', '', cleaned_text)
    # Remove mentions
    cleaned_text = re.sub('@\S+', ' ', cleaned_text)
    # Remove punctuations
    cleaned_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleaned_text)
    # Remove non-ASCII characters
    cleaned_text = re.sub(r'[^\x00-\x7f]', r' ', cleaned_text)
    # Replace multiple spaces with a single space
    cleaned_text = re.sub('\s+', ' ', cleaned_text)
    # Remove leading and trailing whitespaces
    cleaned_text = cleaned_text.strip()

    return cleaned_text

In [9]:
df['cleaned_resume'] = df['Resume'].apply(lambda x:preprocess_text(x))
df.head()

,Category,Resume,cleaned_resume
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas num...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details May 2013 to May 2017 B E UIT...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...


'cleaned_resume' is the result if cleaning the dataset.

##import job description


because our test job description is in pdf format, we install PyPDF2 to read the pdf file.

In [10]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [11]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [12]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.0 MB/s eta 0:00:00


In [13]:
import PyPDF2, pdfplumber

In [14]:
CV="../input/DATAPDF/CV.pdf"
Req="description.pdf"

In [15]:
Req_File=open(Req,'rb')
Script_Req=PyPDF2.PdfReader(Req_File)
#len(Script_Req.pages)
pages=len(Script_Req.pages)


we extract the text from the job description pdf file and populate our empty array


In [16]:
Script_Req = []
with pdfplumber.open(Req_File) as pdf:
    for i in range (0,pages):
        page=pdf.pages[i]
        text=page.extract_text()
        print (text)
        Script_Req.append(text)

JOB DESCRIPTION | Data Analyst
Job Title: Data Analyst Salary Grade (HR):
Job Code: FLSA Status (HR): Non-Exempt
Department Name: Information Technology and Data Approved By (HR):
Reports to (Title): Chief Operations Officer Date Approved (HR):
Date Prepared: October 11, 2016 Prepared By: Teresa Mooney
JOB SUMMARY: Under the supervision of the Chief Operations Officer, thi
s vital role supports HOPES
strategic business plan, agency data and reporting initiatives. This po
sition has a specific focus on
developing and coordinating systems and processes that engage and solic
it data management and
reporting analytics across the agency.
ESSENTIAL FUNCTIONS:
Leadership | Identify, Evaluate & Utilizes Data | Data Project Manageme
nt | Training & Development of
Data Material
Leadership
1. Ability to capture vision, utilize great communication skills to co
llaborate and partner with
multidisciplinary team members to help launch strategic data driven ini
tiatives.
2. Work closely with CEO and C

now that we were able to extract the text, now we have to turn the whole array into a a string so we can convert into a vector to be compared.

In [17]:
Script_Req=''.join(Script_Req)
Req_Clear=Script_Req.replace("\n","")
Req_Clear

'JOB DESCRIPTION | Data AnalystJob Title: Data Analyst Salary Grade (HR):Job Code: FLSA Status (HR): Non-ExemptDepartment Name: Information Technology and Data Approved By (HR):Reports to (Title): Chief Operations Officer Date Approved (HR):Date Prepared: October 11, 2016 Prepared By: Teresa MooneyJOB SUMMARY: Under the supervision of the Chief Operations Officer, this vital role supports HOPESstrategic business plan, agency data and reporting initiatives. This position has a specific focus ondeveloping and coordinating systems and processes that engage and solicit data management andreporting analytics across the agency.ESSENTIAL FUNCTIONS:Leadership | Identify, Evaluate & Utilizes Data | Data Project Management | Training & Development ofData MaterialLeadership1. Ability to capture vision, utilize great communication skills to collaborate and partner withmultidisciplinary team members to help launch strategic data driven initiatives.2. Work closely with CEO and COO on business plan/ 

In [18]:
Match_Test=[df['cleaned_resume'][40],Req_Clear]

df['clean_resume'][40] is one of the CVs from our set of CVs

Req_Clear is a job description

use CountVectorizer to tokenize our text and vectorize it so we can be able to calculate the match percentage

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
count_matrix=cv.fit_transform(Match_Test)

use cosine_similarity to measure similarities between vectors i.e CV and Description

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
print('Similarity is :',cosine_similarity(count_matrix))

Similarity is : [[1.         0.05671155]
 [0.05671155 1.        ]]


In [56]:
df['cleaned_resume'][40]

'TECHNICAL SKILLS Typewriting TORA SPSSEducation Details January 2017 MBA Chidambaram Tamil Nadu SNS College of Engineering January 2014 HSC at SAV Higher Secondary School MBA SNS College of Engineering SSLC Finance at Kamaraj Matriculation School HR Skill Details Human resource Finance Exprience Less than 1 year monthsCompany Details company description'

on the above CV example, we have a 5.6% match.  this makes sense because this HR CV is compared to a data analyst job description





In [21]:
len(df['cleaned_resume'])

962

In [24]:
for i in range( len(df['cleaned_resume'])) :
  df['ID'][i] = 'CV'+ str(i)



df.head()


,Category,Resume,cleaned_resume,ID
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas num...,CV0
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details May 2013 to May 2017 B E UIT...,CV1
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...,CV2
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills R Python SAP HANA Tableau SAP HANA SQL ...,CV3
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...,CV4


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(df['cleaned_resume'])) :
  Match_Test=[df['cleaned_resume'][i],Req_Clear]

  from sklearn.feature_extraction.text import CountVectorizer
  cv=CountVectorizer()
  count_matrix=cv.fit_transform(Match_Test)
  Match=cosine_similarity(count_matrix)[0][1]*100
  df['Percentage'][i] = round(Match)

  #print(": %s      matching: %s"  % (df['Category'][i],round(Match) ) + "%")




In [27]:
df

,Category,Resume,cleaned_resume,ID,Percentage
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas num...,CV0,57
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details May 2013 to May 2017 B E UIT...,CV1,26
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...,CV2,36
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills R Python SAP HANA Tableau SAP HANA SQL ...,CV3,59
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...,CV4,8
...,...,...,...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...,Computer Skills Proficient in MS office Word B...,CV957,46
958,Testing,â Willingness to accept the challenges. â ...,Willingness to a ept the challenges Positive t...,CV958,60
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",PERSONAL SKILLS Quick learner Eagerness to lea...,CV959,44
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...,CV960,7


In [28]:
df[df['ID'] == 'CV817']

,Category,Resume,cleaned_resume,ID,Percentage
817,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV817,71


In [29]:
df.sort_values(by=['Percentage'], ascending=False)

,Category,Resume,cleaned_resume,ID,Percentage
817,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV817,71
802,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV802,71
807,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV807,71
797,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV797,71
812,ETL Developer,SKILL SET â Talend Big Data â Informatica ...,SKILL SET Talend Big Data Informatica Power ce...,CV812,71
...,...,...,...,...,...
559,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...,TECHNICAL PROFICIENCIES Platform Ubuntu Fedora...,CV559,0
583,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...,TECHNICAL PROFICIENCIES Platform Ubuntu Fedora...,CV583,0
553,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...,TECHNICAL PROFICIENCIES Platform Ubuntu Fedora...,CV553,0
589,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...,TECHNICAL PROFICIENCIES Platform Ubuntu Fedora...,CV589,0


In [30]:
df.describe()

,Percentage
count,962.000000
mean,37.347193
std,17.985948
min,0.000000
25%,23.000000
50%,40.000000
75%,52.000000
max,71.000000


#Compare Role to CV Match

df['clean_resume'][40] is one of the CVs from our set of CVs

Req_Clear is a job description

In [31]:
Match_Test=[df['cleaned_resume'][40],Req_Clear]

use CountVectorizer to tokenize our text and vectorize it so we can be able to calculate the match percentage

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
count_matrix=cv.fit_transform(Match_Test)

use cosine_similarity to measure similarities between vectors i.e CV and Description

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
print('Similarity is :',cosine_similarity(count_matrix))

Similarity is : [[1.         0.05671155]
 [0.05671155 1.        ]]


for the purposes of time, we limited our job descriptions to 6000 descriptions to compare 1 CV to.

In [37]:
jdescrip = pd.read_csv('job_postings.csv')
jdescrip.head()
jdescrip = jdescrip[['description']].copy()
jdescrip = jdescrip[jdescrip.index <=6000]

jdescrip.head()


,description
0,While many industries were hurt by the last fe...
1,Are you a dynamic and creative marketing profe...
2,Join Us as a Model Risk Auditor – Showcase You...
3,Business ManagerFirst Baptist Church ForneyFor...
4,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...


In [38]:
jdescrip['description'].info()

<class 'pandas.core.series.Series'>
Int64Index: 6001 entries, 0 to 6000
Series name: description
Non-Null Count  Dtype 
--------------  ----- 
6001 non-null   object
dtypes: object(1)
memory usage: 93.8+ KB


In [39]:
jdescrip['description'][2]

"Join Us as a Model Risk Auditor – Showcase Your Financial Services Auditing Expertise!Are you an experienced financial services auditor with a passion for unraveling complexities? We're on the lookout for a Model Risk Auditor to join us and elevate our model risk management practices.\nPosition Highlights:Location: New York, NY (Hybrid work arrangement)Duration: Approximately 10 weeks (3 months)Daily Rate: Competitive rate based on industry research\nPosition Summary:As a Model Risk Auditor, you will be an integral part of our Model Risk Management (MRM) audit assignment. You will work closely with our Internal Audit team, following our established methodology to assess and evaluate the effectiveness of our model risk management framework. This includes areas such as Model Governance, Policies and Procedures, and key processes and controls in accordance with regulatory guidelines (SR 11-7).\nKey Responsibilities:Collaborate with the lead auditor to plan and execute the MRM audit assig

In [40]:
Script_Req=''.join(jdescrip['description'][2])
Req_Clear=Script_Req.replace("\n","")
Req_Clear

"Join Us as a Model Risk Auditor – Showcase Your Financial Services Auditing Expertise!Are you an experienced financial services auditor with a passion for unraveling complexities? We're on the lookout for a Model Risk Auditor to join us and elevate our model risk management practices.Position Highlights:Location: New York, NY (Hybrid work arrangement)Duration: Approximately 10 weeks (3 months)Daily Rate: Competitive rate based on industry researchPosition Summary:As a Model Risk Auditor, you will be an integral part of our Model Risk Management (MRM) audit assignment. You will work closely with our Internal Audit team, following our established methodology to assess and evaluate the effectiveness of our model risk management framework. This includes areas such as Model Governance, Policies and Procedures, and key processes and controls in accordance with regulatory guidelines (SR 11-7).Key Responsibilities:Collaborate with the lead auditor to plan and execute the MRM audit assignment.

In [41]:
cv=CountVectorizer()
count_matrix=cv.fit_transform(Match_Test)

In [42]:
print('Similarity is :',cosine_similarity(count_matrix))

Similarity is : [[1.         0.05671155]
 [0.05671155 1.        ]]


In [46]:
for i in range( len(jdescrip['description'])) :
  jdescrip['Job_ID'] = 'Job_ID'+ str(i)
  jdescrip['Percentage'] = round(i)

for i in range( len(jdescrip['description'])) :
  jdescrip['Job_ID'][i] = 'Job_ID'+ str(i)



jdescrip.head()

,description,Job_ID,Percentage
0,While many industries were hurt by the last fe...,Job_ID0,6000
1,Are you a dynamic and creative marketing profe...,Job_ID1,6000
2,Join Us as a Model Risk Auditor – Showcase You...,Job_ID2,6000
3,Business ManagerFirst Baptist Church ForneyFor...,Job_ID3,6000
4,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,Job_ID4,6000


job descriptions are cleaned and the job descriptions are being compared to the CV

In [50]:
df['cleaned_resume'][2]

for i in range(6001) :
  Script_Req=''.join(jdescrip['description'][i])
  Req_Clear=Script_Req.replace("\n","")
  Match_Test=[df['cleaned_resume'][2],Req_Clear]

  from sklearn.feature_extraction.text import CountVectorizer
  cv=CountVectorizer()
  count_matrix=cv.fit_transform(Match_Test)
  Match=cosine_similarity(count_matrix)[0][1]*100
  jdescrip['Percentage'][i] = round(Match)


In [51]:
jdescrip

,description,Job_ID,Percentage
0,While many industries were hurt by the last fe...,Job_ID0,27
1,Are you a dynamic and creative marketing profe...,Job_ID1,35
2,Join Us as a Model Risk Auditor – Showcase You...,Job_ID2,30
3,Business ManagerFirst Baptist Church ForneyFor...,Job_ID3,29
4,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,Job_ID4,30
...,...,...,...
5996,Good Shot Golf and Sporting Clays is looking f...,Job_ID5996,23
5997,Business Systems Analyst (Spanish required)Thi...,Job_ID5997,36
5998,Nature and Scope of JobThe BMS Integrator / Fr...,Job_ID5998,22
5999,Channel Account Executive - Latin America (Spa...,Job_ID5999,37


job descriptions are ranked in descending order

In [55]:
jdescrip.sort_values(by=['Percentage'], ascending=False).head(100)

,description,Job_ID,Percentage
3604,The Software Engineering (SE) Analyst II will ...,Job_ID3604,46
3883,Compensation Details\n$19.50 per hour\nWhy sho...,Job_ID3883,45
3886,iHeartMedia\nCurrent employees and contingent ...,Job_ID3886,45
4469,At Dominion Energy we love our jobs. That’s ri...,Job_ID4469,45
2931,ABOUT YOU\nReady to experience a better way to...,Job_ID2931,45
...,...,...,...
1137,About Loxo@Lilly:Loxo@Lilly aims to create med...,Job_ID1137,42
4550,My Client is a premier e-commerce company dedi...,Job_ID4550,42
2885,My client is an industry leader in the Youth S...,Job_ID2885,42
4089,About AscendionAscendion is a full-service dig...,Job_ID4089,42


In [53]:
df['cleaned_resume'][2]

'Areas of Interest Deep Learning Control System Design Programming in Python Electric Machinery Web Development Analytics Technical Activities q Hindustan Aeronautics Limited Bangalore For 4 weeks under the guidance of Mr Satish Senior Engineer in the hangar of Mirage 2000 fighter aircraft Technical Skills Programming Matlab Python and Java LabView Python WebFrameWork Django Flask LTSPICE intermediate Languages and and MIPOWER intermediate Github GitBash Jupyter Notebook Xampp MySQL Basics Python Software Packages Interpreters Anaconda Python2 Python3 Pycharm Java IDE Eclipse Operating Systems Windows Ubuntu Debian Kali Linux Education Details January 2019 B Tech Electrical and Electronics Engineering Manipal Institute of Technology January 2015 DEEKSHA CENTER January 2013 Little Flower Public School August 2000 Manipal Academy of Higher DATA SCIENCE DATA SCIENCE AND ELECTRICAL ENTHUSIAST Skill Details Data Analysis Exprience Less than 1 year months excel Exprience Less than 1 year mon

In [54]:
jdescrip.describe()

,Percentage
count,6001.000000
mean,32.711548
std,6.766408
min,0.000000
25%,30.000000
50%,34.000000
75%,37.000000
max,46.000000
